# Study Notes Chatbot (Prototype 1)

An AI assistant that answers questions from your study materials using RAG (Retrieval-Augmented Generation).

**What it does:**
- Reads multiple PDF documents (lecture notes, textbooks, etc.)
- Answers questions based on document content
- Provides source citations

**Technology:**
- FLAN-T5 for answer generation
- Sentence transformers for semantic search
- FAISS for efficient retrieval

---

## 1. Kaggle Setup

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/study-notes/Lectures/Digital Control Systems/Lecture2.pdf
/kaggle/input/study-notes/Lectures/Digital Control Systems/Lecture3.pdf
/kaggle/input/study-notes/Lectures/Digital Control Systems/Lecture5.pdf
/kaggle/input/study-notes/Lectures/Digital Control Systems/Lecture1.pdf
/kaggle/input/study-notes/Lectures/Digital Control Systems/Lecture4.pdf
/kaggle/input/study-notes/Lectures/Human-Computer Interaction/Lec 3 _  Analyzing and Synthesizing.pdf
/kaggle/input/study-notes/Lectures/Human-Computer Interaction/Lec 1 _ Introduction.pdf
/kaggle/input/study-notes/Lectures/Human-Computer Interaction/Lec 2 _ Needfinding.pdf
/kaggle/input/study-notes/Lectures/Human-Computer Interaction/Lec 5 _ Prototyping and Low-Fidelity Prototypes.pdf
/kaggle/input/study-notes/Lectures/Human-Computer Interaction/Lec 4 _ Design Principles.pdf
/kaggle/input/study-notes/Lectures/Computer Networks/Chapter 4 Easy Subnetting.pdf
/kaggle/input/study-notes/Lectures/Computer Networks/Chapter 2_ TCP_IP.pdf
/

---

## 2. Installing Required Packages

In [2]:
!pip install transformers==4.52.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 79.7 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 38.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.


In [3]:
!pip install sentence-transformers PyPDF2 faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 49.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.9 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━

---

## 3. Importing Packages

Importing libraries for PDF processing, embeddings, and language model inference.

In [4]:
import os
import numpy
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [5]:
import faiss
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer

2025-11-03 02:06:39.054036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762135599.242042      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762135599.296125      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


--- 

## 4. Helper Functions

Core functions for:
- Extracting text from PDFs
- Chunking documents with overlap
- Creating embeddings and vector database
- Semantic search for relevant content

In [176]:
# Wrapping a function for Viz Output
def generate_text_t5(prompt, max_length=512):
    inputs = tokenizer(prompt, return_tensors="pt", max_length=625, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        min_length=200,
        temperature=0.4,
        do_sample=True,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.4,
        no_repeat_ngram_size=4,
        early_stopping=True,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [7]:
def extract_text_from_pdf(pdf_path):
    """Extract text from a single PDF file"""
    try:
        reader = PdfReader(pdf_path)
        full_text = ""
        for page in reader.pages:
            text = page.extract_text()
            if text:
                full_text += text + "\n"
        return full_text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""

In [8]:
def chunk_text(text, chunk_size=500, overlap=100):
    """Split text into overlapping chunks"""
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        if chunk.strip():  # Only add non-empty chunks
            chunks.append(chunk)
    return chunks

In [151]:
import os, re

def load_all_pdfs_from_folder(folder_path, chunk_size=350, overlap=40):
    """
    Load all PDFs from a folder (and subfolders), extract text, chunk, and track metadata.
    
    Returns:
        all_chunks: List of text chunks
        metadata: List of dicts with filename, subject, chunk_id, source
    """
    all_chunks = []
    metadata = []
    
    print(f"📂 Scanning folder: {folder_path}")
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_path = os.path.join(root, file)
                print(f"   📄 Loading: {file}")
                
                # --- Extract text safely ---
                try:
                    text = extract_text_from_pdf(pdf_path)
                except Exception as e:
                    print(f"      ❌ Error reading {file}: {e}")
                    continue
                
                text = re.sub(r'\s+', ' ', text).strip()
                if not text:
                    print(f"      ⚠️ Warning: No text extracted from {file}")
                    continue
                
                # --- Chunk the text ---
                chunks = chunk_text(text, chunk_size=chunk_size, overlap=overlap)
                
                # --- Subject from folder name ---
                subject = os.path.basename(root) if root != folder_path else "General"
                
                # --- Collect metadata ---
                for idx, chunk in enumerate(chunks):
                    all_chunks.append(chunk)
                    metadata.append({
                        'filename': file,
                        'subject': subject,
                        'chunk_id': idx,
                        'source': pdf_path
                    })
                
                print(f"      ✅ Extracted {len(chunks)} chunks")
    
    print(f"\n📊 Total: {len(all_chunks)} chunks from {len(set([m['filename'] for m in metadata]))} files\n")
    return all_chunks, metadata


In [10]:
def embed_chunks(chunks, model_name='sentence-transformers/all-MiniLM-L6-v2'):
    """Create embeddings for text chunks"""
    print("🔄 Creating embeddings...")
    model = SentenceTransformer(model_name)
    embeddings = model.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
    print("✅ Embeddings created!\n")
    return model, embeddings

In [11]:
def create_faiss_index(embeddings):
    """Create FAISS index for fast similarity search"""
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

In [12]:
def search_index(query, model, index, chunks, metadata, k=5):
    """
    Search for most relevant chunks
    
    Returns:
        results: List of tuples (chunk, metadata_dict, distance)
    """
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    
    results = []
    for idx, dist in zip(indices[0], distances[0]):
        results.append({
            'chunk': chunks[idx],
            'metadata': metadata[idx],
            'distance': float(dist)
        })
    
    return results

---

## 5. Trying it

In [62]:
# Load FLAN-T5 model
model_name = "google/flan-t5-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1614: UserWarning: The following device_map keys do not match any submodules in the model: ['decoder.embed_tokens']
  warnings.warn(


Loading all PDFs from the specified folder, extracting text, and organizing into searchable chunks with metadata tracking.

In [152]:
# Notes calling and chunking them
notes_path = "/kaggle/input/study-notes/Lectures"

chunks, metadata = load_all_pdfs_from_folder(notes_path)

📂 Scanning folder: /kaggle/input/study-notes/Lectures
   📄 Loading: Lecture2.pdf
      ✅ Extracted 2 chunks
   📄 Loading: Lecture3.pdf
      ✅ Extracted 2 chunks
   📄 Loading: Lecture5.pdf
      ⚠️ Warning: No text extracted from Lecture5.pdf
   📄 Loading: Lecture1.pdf
      ✅ Extracted 7 chunks
   📄 Loading: Lecture4.pdf
      ✅ Extracted 1 chunks
   📄 Loading: Lec 3 _  Analyzing and Synthesizing.pdf
      ✅ Extracted 7 chunks
   📄 Loading: Lec 1 _ Introduction.pdf
      ✅ Extracted 6 chunks
   📄 Loading: Lec 2 _ Needfinding.pdf
      ✅ Extracted 6 chunks
   📄 Loading: Lec 5 _ Prototyping and Low-Fidelity Prototypes.pdf
      ✅ Extracted 4 chunks
   📄 Loading: Lec 4 _ Design Principles.pdf
      ✅ Extracted 6 chunks
   📄 Loading: Chapter 4 Easy Subnetting.pdf
      ✅ Extracted 3 chunks
   📄 Loading: Chapter 2_ TCP_IP.pdf
      ✅ Extracted 3 chunks
   📄 Loading: Chapter 1_Internetwirking.pdf
      ✅ Extracted 8 chunks
   📄 Loading: Chapter 3 Subnetting VLSM and Troubleshooting_lec.pdf


In [153]:
# Embedding text chunks
model_embeddings, embeddings = embed_chunks(chunks)

🔄 Creating embeddings...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Embeddings created!



In [154]:
# Creating the vector database
index = create_faiss_index(embeddings)

In [179]:
import pprint

question = "What is an OSI model in computer networks?"
top_chunks = search_index(question, model_embeddings, index, chunks, metadata, k=3)

pp = pprint.PrettyPrinter(indent=2, width=100)  # optional: adjust width for wrapping

for i, chunk in enumerate(top_chunks, 1):
    print(f"\n--- Chunk {i} ---")
    pp.pprint(chunk)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- Chunk 1 ---
{ 'chunk': '© Wiley Inc. 2006. All Rights Reserved. Internetworking Basics Advantages of using '
           'routers in your network: •Routers don’t forward broadcasts by default. •Routers can '
           'filter the network based on layer -3 (Network layer) information (i.e., IP address). © '
           'Wiley Inc. 2006. All Rights Reserved. Internetworking Basics Broadcast domains ?? '
           'Collision Domains ?? © Wiley Inc. 2006. All Rights Reserved. Internetworking Models '
           '•The OSI Reference Model –It describes how data and network information are '
           'communicated from an application on one computer, through the network media, to an '
           'application on another computer •The Layered Approach © Wiley Inc. 2006. All Rights '
           'Reserved. Advantages of OSI layered model –Allows multiple -vendor development through '
           'standardization of network components –It divides the network communication process '
         

In [180]:
context = "\n\n".join([r['chunk'] for r in top_chunks])

In [181]:
prompt = f"""
Read the context below and answer the question clearly and concisely.

Guidelines:
- Focus only on key and accurate information.
- Combine repeated ideas into one.
- Organize the answer in short paragraphs or bullet points if appropriate.
- Skip irrelevant text or citations.
- Keep a neutral and professional tone.

Context:
{context}

Question:
{question}

Answer:
"""


In [184]:
answer = generate_text_t5(prompt)

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [185]:
print(answer)

Wiley Inc. 2006. All Rights Reserved. Internetworking Basics Broadcast domains ? Collision Domains ?  Wiley Inc, 2006. All Rights Reserved. Internetwork Models –The OSI Reference Model –It describes how data and network information are communicated from an application on one computer, through the network media, to an application on another computer •The Layered Approach –Allows multiple -vendor development through standardization of network components –Il divides the network communication process into smaller and simpler components, and thus aiding component development, design, and troubleshooting. –Allowing various types of network hardware and software to communicate. –Prevents changes in one layer from affecting other layers, so it does not hamper development. –Improves component development,design, and troubleshoting. –Protects various types of networks hardware and software from each other. –Increases the flexibility of network designs. –Requires less memory and less processing p